# NLP??

In [ ]:
import spacy
from spacy import displacy
from copy import deepcopy
from sec_certs.dataset import CCDataset
from sec_certs.utils.extract import load_text_file, flatten_matches
from sec_certs.cert_rules import cc_rules

In [ ]:
nlp = spacy.load("en_core_web_lg")
default_tokenizer = deepcopy(nlp.tokenizer)
dset = CCDataset.from_json("../../cc_dset/CommonCriteria_dataset.json")
#dset._extract_report_keywords(True)
#dset._extract_targets_keywords(True)
#dset._compute_normalized_cert_ids()
#dset.to_json()

In [ ]:
def customize_tokenizer(cert):
    # The following groups of matches will be added to the default tokenizer as special rules (to not split them).
    token_rule_groups = list(cc_rules.keys())
    token_rule_groups.remove("certification_process")
    # Copy the default tokenizer
    tokenizer = deepcopy(default_tokenizer)
    for rule_group in token_rule_groups:
        matches = set()
        if cert.pdf_data.report_keywords:
            matches.update(flatten_matches(cert.pdf_data.report_keywords[rule_group]).keys())
        if cert.pdf_data.st_keywords:
            matches.update(flatten_matches(cert.pdf_data.st_keywords[rule_group]).keys())
        for match in matches:
            tokenizer.add_special_case(match, [{"ORTH": match}])
    tokenizer.add_special_case("re-certification", [{"ORTH": "re-certification"}])
    return tokenizer

In [ ]:
cert = dset["05726637bd47a762"]
txt = load_text_file(cert.state.report_txt_path)[0]
id_matches = set(flatten_matches(cert.pdf_data.report_keywords["cc_cert_id"]).keys())
print(cert.pdf_data.report_keywords["cc_cert_id"])

In [ ]:
nlp.tokenizer = customize_tokenizer(cert)
doc = nlp(txt)

In [ ]:
sent = None
for token in doc:
    if token.text in id_matches:
        if "Certification Report" in token.sent.text:
            continue
        print(token.sent)
        for chunk in token.sent.noun_chunks:
            print("-----")
            print("|", chunk,  "|", spacy.explain(chunk.root.dep_), "|", chunk.root.head.text)
        print("----------------------------------------------------------")

In [ ]:
sent = """The underlying hardware platform, certified by BSI under reference BSI-DSZ-CC-1040 (BSI-DSZ-CC-
1040-2019-MA-01), was re-evaluated by a different ITSEF while remaining under the same scheme
but given a new reference, i.e. BSI-DSZ-CC-1136."""
sent = "This is a re-certification based on BSI-DSZ-CC-1033-2019."
sent = "This evaluation is a re-evaluation of the certified OTA software 3AQ 21530 XAAA version 2.9 and hardware 3AQ 21564 AAAA ICS5A. Certification Report Identifier is SERTIT-003 CR, issue 1.0, 19."